In [18]:
from string import ascii_letters

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Flatten, CuDNNLSTM, Bidirectional, Dropout
from keras.utils import to_categorical

# Gemsim
import gensim.models
from gensim import utils

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score

# from tensorflow.keras.datasets import imdb
# from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

In [19]:
print(tf.config.list_physical_devices('GPU'))

[]


In [20]:
class MyCorpus:
        """An iterator that yields sentences (lists of str)."""
        def __init__(self, text_list: list):
            self.text_list = text_list

        def __iter__(self):
            for line in self.text_list:
                yield line

In [33]:
# Read the data
df = pd.read_csv('../features/web2py_zhenhao.csv')
print(df.shape)
# df

(17895, 3)


In [34]:
# Split data into train and test sets
X = df.context
X = [list(map(lambda y: str(ascii_letters.index(y)), list(x))) for x in X]
y = df.contains_logging
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=0)

# one hot encode outputs, no longer used
y_train_categorical = to_categorical(y_train)
# y_test_categorical = to_categorical(y_test)
# count_classes = y_test_categorical.shape[1]
# assert count_classes == 2

# Word2Vec Model
sentences = MyCorpus(X)
gensim_model = gensim.models.Word2Vec(sentences=sentences, min_count=1)
actual_vocab_size = len(gensim_model.wv.key_to_index)

# Pad the context prep
X_train_party = np.array([list(x) for x in X_train], dtype=object)
X_test_party = np.array([list(x) for x in X_test], dtype=object)

In [23]:
vocab_size = 30
output_dims = 100
max_length = 80
num_epochs = 100
batch_size = 24
dropout = 0.2
trainable = False

In [24]:
# Name, max_length, vocab_size, batch_size
# Todo: output_dims?

# iterations = [
#     ("Standard", 80, 30, 24),
#     ("len70", 70, 30, 24),
#     ("len60", 60, 30, 24),
#     ("len50", 50, 30, 24),
#     ("len40", 40, 30, 24),
#     ("len30", 30, 30, 24),
#     ("Actual_vocab_size", 80, actual_vocab_size, 24)
# ]

# iterations = [
#     ("Actual_vocab_size", 80, actual_vocab_size, 24),
#     ("Actual_vocab_size2", 80, actual_vocab_size, 24),
#     ("Actual_vocab_size3", 80, actual_vocab_size, 24),
#     ("Actual_vocab_size4", 80, actual_vocab_size, 24),
# ]

# iterations = [
#     ("trainable", 80, actual_vocab_size, 24),
#     ("len20", 20, actual_vocab_size, 24),
#     ("len25", 25, actual_vocab_size, 24),
#     ("len90", 90, actual_vocab_size, 24),
#     ("len110", 110, actual_vocab_size, 24),
#     ("len150", 150, actual_vocab_size, 24),
# ]

# Name, max_length, vocab_size, batch_size, trainable, dropout

# iterations = []
# for drop in [0.0, 0.2]:
#     for train in [False, True]:
#         name = ("" if drop else "No") + "Drop_" + ("" if train else "No") + "Train"
#         entry = (name, 80, 30, 24, train, drop)
#         iterations.append(entry)

# Name, max_length, vocab_size, batch_size, trainable, dropout, sigmoid_output

# iterations = [
#     ('Sigmoid', 80, 30, 24, False, 0.2, True)
# ]

# iterations = [
#     ('Two_LSTM_Layers', 80, 30, 24, False, 0.2, True),
#     ('Two_CuDNNLSTM_Layers', 80, 30, 24, False, 0.2, True),
#     ('Two_CuDNNLSTM+dense_relu', 80, 30, 24, False, 0.2, True)
# ]

# iterations = [
#     ('bidirectional', 80, 30, 24, False, 0.2, True),
#     ('bidirectional', 80, 30, 48, False, 0.2, True),
# ]

iterations = [
    ('Dropout', 80, 30, 24, False, 0.2, True),
]

# iterations = [
#     ('Two_CuDNNLSTM+dense_relu', 80, 30, 24, True, 0.2, True),
#     ('Two_LSTM_Layers+dense_relu', 80, 30, 24, False, 0.2, True),
#     ('Two_LSTM_Layers+dense_relu', 80, 30, 24, True, 0.2, True),
# ]

# Name, max_length, vocab_size, batch_size, trainable, dropout, sigmoid_output, val_split, callback

# iterations = [
#     ('Two_Cu+de|val_split', 80, 30, 24, False, 0.2, True, 0.0, False),
#     ('Two_Cu+de|val_split', 80, 30, 24, False, 0.2, True, 0.1, False),
#     ('Two_Cu+de|val_split', 80, 30, 24, False, 0.2, True, 0.2, False),
# ]



# iterations = []
# for batch_size in [16,20,28,32,50,100]:
#     iterations.append((f'batch{batch_size}', 80, 30, batch_size, True, 0.2, True))


all_scores = []
iterations

[('Sigmoid', 80, 30, 24, False, 0.2, True)]

In [25]:
out = open("results.txt", "a")
# out.write("Name, max_length, vocab_size, batch_size, Balanced_Accuracy, Precision_Score, Recall_Score, F1_Score")
out.write("Name, max_length, vocab_size, batch_size, trainable, dropout, sigmoid_out, Bal_Acc, Precision, Recall, F1_Score")
# out.write("Name, max_length, vocab_size, batch_size, trainable, dropout, sigmoid_out, val_split, callback, Bal_Acc, Precision, Recall, F1_Score")
out.write("\n")
out.close()

In [26]:
callback = False
for name, max_length, vocab_size, batch_size, trainable, dropout, sigmoid in iterations:
# for name, max_length, vocab_size, batch_size, trainable, dropout, sigmoid, val_split, callback in iterations:
    print(name)
    # Build embedding matrix
    # Shouldn't be in the loop as varying vocab_size doesn't help
    embedding_matrix = zeros((vocab_size, output_dims))
    for i in range(vocab_size):
        embedding_vector = None
        try:
            embedding_vector = gensim_model.wv[str(i)]
        except KeyError:
            pass
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    # Pad the context
    padded_inputs = pad_sequences(X_train_party, maxlen=max_length, value=0.0)  # 0.0 because it corresponds with <PAD>
    padded_inputs_test = pad_sequences(X_test_party, maxlen=max_length, value=0.0)  # 0.0 because it corresponds with <PAD>

    # Build the model
    model = Sequential()
    model.add(Embedding(vocab_size, output_dims, weights=[embedding_matrix], input_length=max_length, trainable=trainable))

    # if name == "Two_LSTM_Layers":
    #     model.add(keras.layers.LSTM(128, dropout=dropout, return_sequences=True))
    #     model.add(keras.layers.LSTM(128, dropout=dropout))
    # elif name == "Two_LSTM_Layers+dense_relu":
    #     model.add(keras.layers.LSTM(128, dropout=dropout, return_sequences=True))
    #     model.add(keras.layers.LSTM(128, dropout=dropout))
    #     model.add(Dense(32,activation='relu'))
    # elif name == "Two_CuDNNLSTM_Layers":
    #     model.add(CuDNNLSTM(128, return_sequences=True))
    #     model.add(CuDNNLSTM(128))
    # elif name == "Two_CuDNNLSTM+dense_relu":
    #     model.add(CuDNNLSTM(128, return_sequences=True))
    #     model.add(CuDNNLSTM(128))
    #     model.add(Dense(32,activation='relu'))
    # else:
    #     model.add(keras.layers.LSTM(128, dropout=dropout))

    # Best so far:   (Add dropout layer?)
    if name == "bidirectional":
        model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
        model.add(Bidirectional(CuDNNLSTM(128)))
    else:
        model.add(CuDNNLSTM(128, return_sequences=True))
        model.add(CuDNNLSTM(128))
    model.add(Dense(32,activation='relu'))

    # Dropout with layer, terrible
    # model.add(keras.layers.LSTM(128))
    # model.add(keras.layers.Dropout(dropout))

    if sigmoid:
        model.add(Dense(1, activation='sigmoid'))
    else:
        model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    if callback:
        # es = EarlyStopping(monitor='loss', mode='auto', verbose=1, patience=4)
        es = EarlyStopping(monitor='accuracy', mode='auto', verbose=1, patience=4)
        model.fit(padded_inputs, y_train if sigmoid else y_train_categorical,
                  epochs=num_epochs, batch_size=batch_size, validation_split=val_split, callbacks=[es])
    else:
        model.fit(padded_inputs, y_train if sigmoid else y_train_categorical,
                  epochs=num_epochs, batch_size=batch_size,
                  # validation_split=val_split
                  )

    # Predict
    # pred_train= model.predict(padded_inputs)
    pred_test= model.predict(padded_inputs_test)
    if sigmoid:
        y_pred = np.round(pred_test)
    else:
        y_pred = []
        for zero, one in pred_test:
            if zero > 0.5:
                y_pred.append(0)
            else:
                y_pred.append(1)

    # Scores
    scores = [
        name,
        max_length,
        vocab_size,
        batch_size,
        trainable,
        dropout,
        sigmoid,
        # val_split,
        # callback,
        f"{balanced_accuracy_score(y_test, y_pred):.2f}"[2:],
        f"{precision_score(y_test, y_pred):.2f}"[2:],
        f"{recall_score(y_test, y_pred):.2f}"[2:],
        f"{f1_score(y_test, y_pred):.2f}"[2:],
    ]
    out = open("results.txt", "a")
    out.write(str(scores).replace("'", "")[1:-1])
    out.write("\n")
    out.close()
    all_scores.append(scores)

Sigmoid
Epoch 1/2
560/560 [==============================] - 27s 43ms/step - loss: 0.0922 - accuracy: 0.9817
Epoch 2/2
140/140 [==============================] - 3s 21ms/step


/usr/local/Cellar/python@3.9/3.9.0_2/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# print("vocab_size =", vocab_size)
# print("output_dims =", output_dims)
# print("max_length =", max_length)
# print("num_epochs =", num_epochs)
# print("batch_size =", batch_size)
# print("-------------------")
# print("balanced_accuracy_score:", balanced_accuracy_score(y_test, y_pred))
# print("precision_score:", precision_score(y_test, y_pred))
# print("recall_score:", recall_score(y_test, y_pred))
# print("f1_score:", f1_score(y_test, y_pred))